In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
# Data processing module.
def load_data(data_index,test_size,random_state):
    """
    data_index: 数据集序号（1,2,3）
    pre_index: 预测属性序号（1,2,3）
    test_size: 测试集比例 (0-1)
    注意，python的编号是从0开始。对于区间而言，冒号左右为左闭右开。例如：[:,1:5]取所有行，取第二列到第四列（不包括第五列）。
    """ 
    if(data_index == 1):
        df = pd.read_csv("data1xin.csv")
        X = df.values[:,:25]
        y = df.values[:,25]
    elif(data_index == 2):
        df = pd.read_csv("data2xin.csv")
        X = df.values[:,:21]
        y = df.values[:,21]
    elif(data_index == 3):
        df = pd.read_csv("data3xin.csv")
        X = df.values[:,:16]
        y = df.values[:,16]
    elif(data_index == 4):
        df = pd.read_csv("data4xin.csv")
        X = df.values[:,:13]
        y = df.values[:,13]
    elif(data_index == 5):
        df = pd.read_csv("data5xin.csv")
        X = df.values[:,:12]
        y = df.values[:,12]
    elif(data_index == 6):
        df = pd.read_csv("data6xin.csv")
        X = df.values[:,:10]
        y = df.values[:,10]
    elif(data_index == 222):
        df = pd.read_csv("data222.csv")
        X = df.values[:,:17]
        y = df.values[:,17]
    else:
        print("数据集选择错误！")
        return 0

    poly = PolynomialFeatures(2,include_bias=0)
    #implement polynomial degree=2
    X=poly.fit_transform(X)
    X_n=X
    y_n=y
    X_train_o,X_test_o,y_train_o,y_test_o = train_test_split(X_n,y_n,test_size=test_size,random_state=random_state) # These variables are before gaussian noise.
    #print(y)
    for i in range(10):#-------add gaussian noise. 加入高斯噪声。改变range里的数字可以控制加入噪声的数量。
        X_noise=np.random.normal(0,1,np.shape(X))*0.01
        X_n=np.r_[X_n,X+X_noise]
        y_n=np.r_[y_n,y]
    
    X_train,X_test,y_train,y_test = train_test_split(X_n,y_n,test_size=test_size,random_state=random_state) #These vairables are after gaussian noise.

    return X_train,X_test,y_train,y_test,X,y,X_test_o,y_test_o,X_train_o,y_train_o

In [3]:
# Data visualization module. 数据可视化
def show_2(model,model_name,y_pred1,y_pred2,y_test1,y_test2):
    y_pred_all = np.hstack((y_pred1,y_pred2))
    y_test_all = np.hstack((y_test1,y_test2))

    #plt.style.use("ggplot")
    method=model_name

    y_pred_all = np.hstack((y_pred1,y_pred2))
    y_test_all = np.hstack((y_test1,y_test2))
    RMSE1 = (mean_squared_error(y_pred_all,y_test_all))**0.5
    RMSE2 = (mean_squared_error(y_pred1,y_test1))**0.5

   # print("测试集的RMSE："+str((mean_squared_error(y_pred1,y_test1))**0.5))
    print("总的RMSE："+str((mean_squared_error(y_pred_all,y_test_all)**0.5)))



    # Second plot.

    #print("测试集的R2："+str(r2_score(y_pred1,y_test1)))
    print("总的R2："+str(r2_score(y_pred_all,y_test_all)))
    

In [11]:
#设置模型


models = []
model_names = ["Linear Regression","svm","AdaBoostRegressor",'Gradiant Boosting Regression']

models.append(linear_model.LinearRegression())   #线性回归
#models.append(RandomForestRegressor(min_samples_split=2,max_depth=4,min_samples_leaf=4,n_estimators=300))             #岭回归
#models.append(AdaBoostRegressor(base_estimator=DecisionTreeRegressor(min_samples_split=2,max_depth=4,max_features='sqrt',min_samples_leaf=4),n_estimators=300,learning_rate=0.1))      #贝叶斯岭回归
#models.append(GradientBoostingRegressor(min_samples_split=2,max_depth=2,max_features='sqrt',min_samples_leaf=4,n_estimators=300,learning_rate=0.05)) #min_samples_leaf 越大防止过拟合



X_train,X_test,y_train,y_test,Original_x,Origial_y,X_test_o,y_test_o,X_train_o,y_train_o = load_data(data_index=1,test_size=0.2,random_state=1)
j = 0
for i,model in enumerate(models):
    model.fit(X_train,y_train)
    y_pred1 = model.predict(X_test_o)
    y_pred2 = model.predict(X_train_o)
    y_test1 = y_test_o
    y_test2 = y_train_o
    show_2(model,model_names[j],y_pred1,y_pred2,y_test1,y_test2)
    j+=1
    np.savetxt('LR测试预测1.csv', y_pred1, delimiter=",")
    np.savetxt('LR训练预测1.csv', y_pred2, delimiter=",")
    np.savetxt('LR测试真实1.csv', y_test1, delimiter=",")
    np.savetxt('LR训练真实1.csv', y_test2, delimiter=",")
    

总的RMSE：0.40515993002055783
总的R2：0.32910159662355853


In [14]:
#计算500次平均的RMSE以及R2。根据电脑配置不同，需要的时间可能较长。
models = []
model_names = ["Linear Regression","Ridge Regression","BayesRidge Regression","Lasso Regression",'Gradiant Boosting Regression']

#models.append(linear_model.LinearRegression())   #线性回归
#models.append(linear_model.Ridge())              #岭回归
#models.append(linear_model.BayesianRidge())      #贝叶斯岭回归
#models.append(linear_model.Lasso())              #Lasso回归
models.append(GradientBoostingRegressor(min_samples_split=2,max_depth=2,max_features='sqrt',min_samples_leaf=4,n_estimators=300,learning_rate=0.05)) #min_samples_leaf 越大防止过拟合


# 500 times average statistics. 

for j,model in enumerate(models):
    #All_MSE_temp = []
    All_R2_temp = []
    #All_MAE_temp = []
    #Test_MSE_temp = []
    Test_R2_temp = []
    #Test_MAE_temp = []
    All_RMSE_temp = []
    Test_RMSE_temp = []

    for i in range(1000):
        X_train,X_test,y_train,y_test,Original_x,Origial_y,X_test_o,y_test_o,X_train_o,y_train_o = load_data(data_index=222,test_size=0.2,random_state=0)
        model.fit(X_train,y_train)
        y_pred1 = model.predict(X_test_o)
        y_pred2 = model.predict(X_train_o)
        y_test1 = y_test_o
        y_test2 = y_train_o
        y_pred1 = model.predict(X_test_o)
        y_pred2 = model.predict(X_train_o)
        y_pred_all = np.hstack((y_pred1,y_pred2))
        y_test_all = np.hstack((y_test1,y_test2))
        #All_MSE_temp.append(mean_squared_error(y_pred_all,y_test_all))
        All_R2_temp.append(r2_score(y_pred_all,y_test_all))
        #All_MAE_temp.append(mean_absolute_error(y_pred_all,y_test_all))
        #Test_MSE_temp.append(mean_squared_error(y_pred1,y_test1))
        Test_R2_temp.append(r2_score(y_pred1,y_test1))
        #Test_MAE_temp.append(mean_absolute_error(y_pred1,y_test1))
        All_RMSE_temp.append(mean_squared_error(y_pred_all,y_test_all)**0.5)
        Test_RMSE_temp.append(mean_squared_error(y_pred1,y_test1)**0.5)
        
        np.savetxt('1000RMSE.csv',All_RMSE_temp,delimiter=",")
        np.savetxt('1000R2.csv',All_R2_temp,delimiter=",")
        
    All_RMSE = (sum(All_RMSE_temp)/1000)
    #All_MSE = (sum(All_MSE_temp)/500)
    All_R2 = sum(All_R2_temp)/1000
    #All_MAE = sum(All_MAE_temp)/200
    Test_RMSE = (sum(Test_RMSE_temp)/1000)
    #Test_MSE = (sum(Test_MSE_temp)/200)
    Test_R2 = sum(Test_R2_temp)/1000
    #Test_MAE = sum(Test_MAE_temp)/200
    
    print("模型: "+str(model))
    
    print("总体数据RMSE:{}".format(All_RMSE))
    #print("总体数据MSE:{}".format(All_MSE))
   # print("总体数据MAE：{}".format(All_MAE))
    print("总体数据R2:{}".format(All_R2))
    
    print("测试数据RMSE:{}".format(Test_RMSE))
   # print("测试集数据MSE:{}".format(Test_MSE))
   # print("测试集数据MAE：{}".format(Test_MAE))
    print("测试集数据R2:{}".format(Test_R2))
    print("\n")
    
    """
    plt.figure(dpi=300,figsize=(10,10))
    plt.style.use("tableau-colorblind10")
    name = [u"总体数据MSE",r"$R^2$","总体数据MAE","测试集数据MSE",r"$R^2$",u"测试集数据MAE"]
    plt.bar(name,height=[All_MSE,All_R2,All_MAE,Test_MSE,Test_R2,Test_MAE])
    plt.title("xxx")
    """
    
    

模型: GradientBoostingRegressor(learning_rate=0.05, max_depth=2, max_features='sqrt',
                          min_samples_leaf=4, n_estimators=300)
总体数据RMSE:0.09156756207899679
总体数据R2:0.975112829859849
测试数据RMSE:0.064527470180713
测试集数据R2:0.9881326923152238


